# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
from re import T
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)
tsum = 0
for sqrt_value in generator:
    print(sqrt_value)
    tsum+=sqrt_value
print(tsum)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
8.382332347441762


In [2]:
from re import T
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 13
generator = square_root_generator(limit)
tsum = 0
for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [3]:
import duckdb
conn = duckdb.connect(database=':memory:', read_only=False)

def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}

for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [6]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [14]:
import duckdb

conn = duckdb.connect(database=':memory:', read_only=False)

def people_1():
    for i in range(1, 6):
        yield (i, f"Person_{i}", 25 + i, "City_A", None)

def people_2():
    for i in range(3, 9):
        yield (i, f"Person_{i}", 30 + i, "City_B", f"Job_{i}")

conn.execute("CREATE TABLE people (ID INTEGER, Name VARCHAR, Age INTEGER, City VARCHAR, Occupation VARCHAR)")

conn.executemany("INSERT INTO people VALUES (?, ?, ?, ?, ?)", people_1())

conn.executemany("INSERT INTO people VALUES (?, ?, ?, ?, ?)", people_2())

result = conn.execute("SELECT SUM(Age) FROM people")
sum_of_ages = result.fetchone()[0]
print(sum_of_ages)

353


In [15]:
import duckdb

conn = duckdb.connect(database=':memory:', read_only=False)

def people_1():
    for i in range(1, 3):
        yield (i, f"Person_{i}", 25 + i, "City_A", None)
for person in people_1():
    print(person)
def people_2():
    for i in range(3, 9):
        yield (i, f"Person_{i}", 30 + i, "City_B", f"Job_{i}")
for person in people_2():
    print(person)

conn.execute("CREATE TABLE people2 (ID INTEGER, Name VARCHAR, Age INTEGER, City VARCHAR, Occupation VARCHAR)")

conn.executemany("INSERT INTO people2 VALUES (?, ?, ?, ?, ?)", people_1())

conn.executemany("INSERT INTO people2 VALUES (?, ?, ?, ?, ?)", people_2())

result = conn.execute("SELECT SUM(Age) FROM people2")
sum_of_ages = result.fetchone()[0]
print(sum_of_ages)

(1, 'Person_1', 26, 'City_A', None)
(2, 'Person_2', 27, 'City_A', None)
(3, 'Person_3', 33, 'City_B', 'Job_3')
(4, 'Person_4', 34, 'City_B', 'Job_4')
(5, 'Person_5', 35, 'City_B', 'Job_5')
(6, 'Person_6', 36, 'City_B', 'Job_6')
(7, 'Person_7', 37, 'City_B', 'Job_7')
(8, 'Person_8', 38, 'City_B', 'Job_8')
266
